# Introduction to Fully Homomorphic Encryption


FHE (Fully Homomorphic Encryption)

In [1]:
using Nemo


Welcome to Nemo version 0.6.3

Nemo comes with absolutely no warranty whatsoever



In [2]:
N = 16 # Polynomial size

16

## Utlity functions
Conversion functions between vectors and polynomials: 

In [3]:
function vector2poly(vector, freevar)
    acc = 0
    for (index, value) in enumerate(vector)
        acc = acc + freevar ^(index -1) * value
    end
    return acc
end

function poly2vector(poly)
    size = Nemo.degree(poly) + 1
    result = zeros(base_ring(poly), size)
    for i in 0:1:size-1
        result[i+1] = Nemo.coeff(poly, i)
    end
    return result
end

poly2vector (generic function with 1 method)

In [4]:
""" Random polynomial generation in S """
function random_poly(freevar, degree::Int, coeff_space)
    acc = 0
    for i in 0:1:degree
        acc = acc + rand(coeff_space) * freevar^i
    end
    return acc
end

random_poly

An FHE Cipher is a random vector and a scalar value

In [5]:
type FHECipher
    a
    b
end

In [6]:
""" Learning with error """
function fhe_encode(msg, key)
    a = mod.(rand(N), 0.3)
    b = msg + key'a
    return FHECipher(a, b)
end
function fhe_decode(ciphertext::FHECipher, key)
    phase = ciphertext.a'key
    return mod.(ciphertext.b - phase, 1.0)
end
function fhe_add(c1::FHECipher, c2::FHECipher)
    b = mod.(c1.b + c2.b, 1.0)
    a = mod.(c1.a + c2.a, 1.0)
    return FHECipher(a, b)
end

fhe_add (generic function with 1 method)

In [7]:
msg0 = 0.1
msg1 = 0.3
key = rand(N)

c0 = fhe_encode(msg0, key)
p0  = fhe_decode(c0, key)

c1 = fhe_encode(msg1, key)
p1  = fhe_decode(c1, key)

println("decode(c0 + c1) $(fhe_decode(fhe_add(c0, c1), key))")

decode(c0 + c1) 0.40000000000000036


## Ring Learning With Error

Ring Learning With Error (RLWE) bases homomorphic encryption on a polynomial Ring.


In [8]:
type RLWESystem
    LARGE_PRIME::Nemo.fmpz
    N::Int64
    R
    z
    MOD_POLY
end

type RLWECiphertext
    A
end

function init_RLWE_system(LARGE_PRIME::Nemo.fmpz, N::Int64)
    R, z = PolynomialRing(ResidueRing(ZZ, LARGE_PRIME), "z")
    MOD_POLY = z^N+1    
    return RLWESystem(LARGE_PRIME, N, R, z, MOD_POLY)
end

init_RLWE_system (generic function with 1 method)

In [9]:
toy_system = init_RLWE_system(
    ZZ(22953686867719691230002707821868552601124472329079),
    4
)

# LARGE_PRIME = ZZ(22953686867719691230002707821868552601124472329079) # 32416190071
# N = 4

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] $ at ./deprecated.jl:385 [inlined]
 [3] _hash_integer(::Int64, ::UInt64) at /home/nicolas/.julia/v0.6/Nemo/src/flint/fmpz.jl:117
 [4] hash at ./tuple.jl:295 [inlined]
 [5] hash at ./hashing.jl:5 [inlined]
 [6] hashindex(::Tuple{Nemo.FlintIntegerRing,Nemo.fmpz}, ::Int64) at ./dict.jl:210
 [7] ht_keyindex(::Dict{Tuple{Nemo.Ring,Nemo.RingElem},Nemo.Ring}, ::Tuple{Nemo.FlintIntegerRing,Nemo.fmpz}) at ./dict.jl:322
 [8] haskey at ./dict.jl:505 [inlined]
 [9] Nemo.GenResRing{Nemo.fmpz}(::Nemo.fmpz, ::Bool) at /home/nicolas/.julia/v0.6/Nemo/src/generic/GenericTypes.jl:153
 [10] #ResidueRing#42(::Bool, ::Function, ::Nemo.FlintIntegerRing, ::Nemo.fmpz) at /home/nicolas/.julia/v0.6/Nemo/src/generic/Residue.jl:597
 [11] ResidueRing(::Nemo.FlintIntegerRing, ::Nemo.fmpz) at /home/nicolas/.julia/v0.6/Nemo/src/generic/Residue.jl:594
 [12] init_RLWE_system(::Nemo.fmpz, ::Int64) at ./In[8]:14
 [13] include_string(::String, ::String

RLWESystem(22953686867719691230002707821868552601124472329079, 4, Univariate Polynomial Ring in z over Residue ring of Integer Ring modulo 22953686867719691230002707821868552601124472329079, z, z^4+1)

In [10]:
function poly_mod_coeff(ctx, poly, modulo) #, freevar)
    result = 0
    size = Nemo.degree(poly)
    ROUND_BOUND = ctx.LARGE_PRIME >> 1
    for i in 0:1:size
        coeff = Nemo.coeff(poly, i).data
        # if the coeffs exceeds a certain bound it must be considered as
        # negative and its parity inversed
        if coeff > ROUND_BOUND
            coeff = 1 - (coeff % modulo)
        else
            coeff = coeff % modulo
        end
        result = result + ctx.z^i * coeff
    end
    return result
end

poly_mod_coeff (generic function with 1 method)

In [11]:
@time key = random_poly(toy_system.z, toy_system.N-1, Int64);

  0.031957 seconds (5.61 k allocations: 310.315 KiB)


In [12]:
msg = vector2poly(mod.(rand(Int64, toy_system.N), 2), toy_system.z)

z+1

## Encoding / Decoding

In [13]:
function rlwe_encode(ctx, msg, key, err_bound)
    # random encoding polynomial
    a = random_poly(ctx.z, N-1, Int32)
    # random error
    err = vector2poly(mod.(rand(Int64, ctx.N), err_bound), ctx.z)
    # encrypted message
    b = 2 * err + mulmod(a , key, ctx.MOD_POLY) + msg
    return [b, -a]
end

function rlwe_decode(ctx, ciphertext, key)
    b, a = ciphertext
    phase = mulmod(a, key, ctx. MOD_POLY)
    reduct = b + phase
    msg = poly_mod_coeff(ctx, reduct, 2) 
    return msg
end

rlwe_decode (generic function with 1 method)

In [14]:
@time ciphertext = rlwe_encode(toy_system, msg, key, 1024);

  0.130658 seconds (122.23 k allocations: 6.425 MiB, 4.66% gc time)


In [15]:
@time decoded_pt = rlwe_decode(toy_system, ciphertext, key);


  0.041893 seconds (26.60 k allocations: 1.398 MiB)


In [16]:
msg, decoded_pt, msg == decoded_pt

(z+1, z+1, true)

### Homomorphic Operation: Addition

In [17]:
""" Homomorphic addition of RLWE encoded ciphertext """
function rlwe_add(c1, c2)
    b1, a1 = c1
    b2, a2 = c2
    return [(b1 +b2), (a1 + a2)]
end

rlwe_add

In [18]:
@time rlwe_decode(
    toy_system,
    rlwe_add(
        rlwe_encode(toy_system, toy_system.z+1, key, 128),
        rlwe_encode(toy_system, toy_system.z^2, key, 128)
    ),
    key
)

  0.014994 seconds (5.58 k allocations: 270.626 KiB)


z^2+z+1

## Homomorphic Operation: Multiplication

$ b_0 = \mu_0 + 2 \times err_0 + a_0 \times k $ 

$ b_1 = \mu_1 + 2 \times err_1 + a_1 \times k $ 

$ b_0 \times b_1 = \mu_0 \times \mu_1 + 2 \times (\mu_0 . err_1 + ...) + \mu_1 \times a_0 \times k + \mu_0 \times a_1 \times k + a_1 \times a_0 \times k^2  $ 

$ b_0 \times b_1 - (a_1 \times b_0 + a_0 \times b_1) \times k + a_0 \times a_1 \times k^2 \equiv \mu_0 \times \mu_1 \pmod 2 $ 

In [19]:
function rlwe_mul(ctx, c1, c2)
    b1, a1 = c1
    b2, a2 = c2
    return  [mulmod(b1, b2, ctx.MOD_POLY), (mulmod(a1, b2, ctx.MOD_POLY) + mulmod(a2, b1, ctx.MOD_POLY)), mulmod(a1, a2, ctx.MOD_POLY)]
end

function rlwe_mul_decode(ctx, c1, key)
    b, a1, a2 = c1
    k2 = mulmod(key, key, ctx.MOD_POLY)
    a2k2 = mulmod(a2, k2, ctx.MOD_POLY)
    a1k = mulmod(a1, key, ctx.MOD_POLY)
    result = b + a1k + a2k2
    return poly_mod_coeff(ctx, result, 2)
end

rlwe_mul_decode (generic function with 1 method)

In [20]:
z = toy_system.z
z_ct = rlwe_encode(toy_system, z^2, key, 128)
z_ct2 = rlwe_encode(toy_system, z+1, key, 128)
z2_mul = rlwe_mul(toy_system, z_ct, z_ct2)
z_dec = rlwe_decode(toy_system, z_ct, key)
z_dec2 = rlwe_decode(toy_system, z_ct2, key)
z2_dec = rlwe_mul_decode(toy_system, z2_mul, key)
z_dec, z_dec2, z2_dec # z2_mul, z_dec, z2_dec

(z^2, z+1, z^3+z^2)

Each multiplication add an extra term to the ciphertext. Those leading to a new form of ciphertext: a vector of polynomial.
Addition is performed through element wise vector addition.
Decoding is performed by evaluation the vector as a polynomial with respect to the key variable.
Multiplication requires some extra works:

$b_0, a_0, c_0 \times (b_1, a_1) $

$b_0 = \mu_0 - a_0 \times k - c_0 \times k ^2  + 2 . \epsilon $

$b_1 = \mu_1 - a_1 \times k + 2 . \epsilon' $

$b_0 \times b_1 = \mu_0 \times \mu_1 - k \times (a_0 . \mu_1 + a_1 \times \mu_0) - k^2 (c_0 \times \mu_1 - a_0 \times a_1) + (...) $



In [21]:
function complement!(vector, n, elt)
   append!(vector, [elt for i in 1:1:n])
end
function rlwe_gen_add(ctx, c0, c1)
    s0 = size(c0, 1)
    s1 = size(c1, 1)
    common_size = max(s0, s1)
    c0 = copy(c0)
    c1 = copy(c1)
    complement!(c0, common_size - s0, ctx.R(0))
    complement!(c1, common_size - s1, ctx.R(0))
   if size(c0, 1) == size(c1, 1)
       return [v0 + v1 for (v0, v1) in zip(c0, c1)]
    else
        println("size $(size(c0)), $(size(c1))")
        throw(ArgumentError("size mismatch"))
    end
end

rlwe_gen_add (generic function with 1 method)

In [22]:
function rlwe_gen_decode(ctx, c, key)
   return poly_mod_coeff(
        ctx,
        reduce(+, 0, [
            mulmod(
                value, 
                powmod(key, index - 1, ctx.MOD_POLY),
                ctx.MOD_POLY
            ) for (index, value) in enumerate(c)
            ]),
        2
    )
end

rlwe_gen_decode (generic function with 1 method)

In [23]:
rlwe_gen_decode(toy_system, z2_mul, key)

z^3+z^2

In [24]:
rlwe_gen_decode(toy_system, rlwe_gen_add(toy_system, z2_mul, z_ct2), key)

z^3+z^2+z+1

In [25]:
""" Generic multiplication of arbitrary size ciphertext """
function rlwe_gen_mul(ctx, c0, c1)
    s0 = size(c0, 1)
    s1 = size(c1, 1)
    # computing degree + of c0 x c1
    r_size = (s0 - 1) + (s1 - 1) + 1
    result = [ctx.R(0) for i in 1:1:r_size]
    for (i0, v0) in enumerate(c0)
        for (i1, v1) in enumerate(c1)
            result[i0 + i1 - 1] = result[i0 + i1 - 1] + mulmod(v0, v1, ctx.MOD_POLY)
        end
    end
    return result
end

rlwe_gen_mul

In [26]:
z3_mul = rlwe_gen_mul(toy_system, z2_mul, z_ct2)
z3_dec = rlwe_gen_decode(toy_system, z3_mul, key)

z^2+1

In [ ]:
type RLWEPublicContext
    ctx::RLWESystem
    ONE
    ZERO
    ERROR_BOUND
end

In [44]:
function rlwe_encode_bit(pctx, bit, key)
    return rlwe_encode(pctx.ctx, pctx.ctx.R(bit), key, pctx.ERROR_BOUND)
end
function rlwe_decode_bit(pctx, ciphertext, key)
    poly = rlwe_gen_decode(pctx.ctx, ciphertext, key)
    return poly
end
function rlwe_bit_XOR(pctx, c0, c1)
    return rlwe_gen_add(pctx.ctx, c0, c1)
end
function rlwe_bit_NOT(pctx, c0)
    return rlwe_bit_XOR(pctx, c0, pctx.ONE)
end
function rlwe_bit_AND(pctx, c0, c1)
    return rlwe_gen_mul(pctx.ctx, c0, c1)
end
function rlwe_bit_OR(pctx, c0, c1)
   # c0 OR c1 = NOT(NOT(c0) AND NOT(c1)) 
    return rlwe_bit_NOT(pctx, rlwe_bit_AND(pctx, rlwe_bit_NOT(pctx, c0), rlwe_bit_NOT(pctx, c1)))
end
    

rlwe_bit_OR (generic function with 1 method)

In [45]:
SKEY = random_poly(toy_system.z, toy_system.N-1, Int64)
pctx = RLWEPublicContext(
    toy_system,
    rlwe_encode(toy_system, toy_system.R(1), SKEY, 128),
    rlwe_encode(toy_system, toy_system.R(0), SKEY, 128),
    128
)
    
pbits = [0, 1]
for b0 in pbits
    c0 = rlwe_encode_bit(pctx, b0, SKEY)
    for b1 in pbits
        c1 = rlwe_encode_bit(pctx, b1, SKEY)
        println("$b0 AND $b1 = $(rlwe_decode_bit(pctx, rlwe_bit_AND(pctx, c0, c1), SKEY))")
        println("$b0 OR  $b1 = $(rlwe_decode_bit(pctx, rlwe_bit_OR(pctx, c0, c1), SKEY))")
        println("$b0 XOR $b1 = $(rlwe_decode_bit(pctx, rlwe_bit_XOR(pctx, c0, c1), SKEY))")
    end
end
        

0 AND 0 = 0
0 OR  0 = 0
0 XOR 0 = 0
0 AND 1 = 0
0 OR  1 = 1
0 XOR 1 = 1
1 AND 0 = 0
1 OR  0 = 1
1 XOR 0 = 1
1 AND 1 = 1
1 OR  1 = 1
1 XOR 1 = 0


## Boostrapping

## Modulo Switching

# Torus Learning with Error (TLWE)
## Defining Mathematical objects

In [29]:
S, x = PolynomialRing(RealField(53), "x");;

T = ResidueRing(S, x^N+1);

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] $ at ./deprecated.jl:385 [inlined]
 [3] hash(::Nemo.arb_poly, ::UInt64) at /home/nicolas/.julia/v0.6/Nemo/src/generic/Poly.jl:73
 [4] hash at ./tuple.jl:295 [inlined]
 [5] hash at ./hashing.jl:5 [inlined]
 [6] hashindex(::Tuple{Nemo.ArbPolyRing,Nemo.arb_poly}, ::Int64) at ./dict.jl:210
 [7] ht_keyindex(::Dict{Tuple{Nemo.Ring,Nemo.RingElem},Nemo.Ring}, ::Tuple{Nemo.ArbPolyRing,Nemo.arb_poly}) at ./dict.jl:322
 [8] haskey at ./dict.jl:505 [inlined]
 [9] Nemo.GenResRing{Nemo.arb_poly}(::Nemo.arb_poly, ::Bool) at /home/nicolas/.julia/v0.6/Nemo/src/generic/GenericTypes.jl:153
 [10] #ResidueRing#42(::Bool, ::Function, ::Nemo.ArbPolyRing, ::Nemo.arb_poly) at /home/nicolas/.julia/v0.6/Nemo/src/generic/Residue.jl:597
 [11] ResidueRing(::Nemo.ArbPolyRing, ::Nemo.arb_poly) at /home/nicolas/.julia/v0.6/Nemo/src/generic/Residue.jl:594
 [12] include_string(::String, ::String) at ./loading.jl:522
 [13] include_string(::Module, ::

In [30]:
function tlwe_gen_key()
    return vector2poly(rand(0:1, N))
end
function tlwe_encode(msg, key)
    
end

tlwe_encode (generic function with 1 method)